In [18]:
# Import the dependencies.
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify



In [19]:
#################################################
# Database Setup
#################################################
# Correct path to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite", echo=False)


# reflect an existing database into a new model
Base = automap_base()


In [20]:
# reflect the tables
Base.prepare(autoload_with=engine)

# Save references to each table
Station = Base.classes.station
Measurement = Base.classes.measurement


# Create our session (link) from Python to the DB
session = Session(engine)



In [21]:
#################################################
# Flask Setup
#################################################

app = Flask(__name__)
# What does this do?
# __name__ is a special variable in Python that gets as value the string "__main__" when you're executing the script.
# It evaluates to true when run as a script.
# It evaluates to false when imported as a module.
#


In [22]:
#################################################
# Flask Routes
#################################################
@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation"
        f"/api/v1.0/stations"
        f"/api/v1.0/tobs"
        f"/api/v1.0/<start>"
        f"/api/v1.0/<start>/<end>"
    )


In [23]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Create our session (link) from Python to the DB
    session = Session(engine)
    results = session.query(Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs).all()

    session.close()
    
    # Create a dictionary from the row data and append to a list of all_passengers
    all_precipitations = []
    for station, date, prcp, tobs in results:
        precipitations_dict = {}
        precipitations_dict["station"] = station
        precipitations_dict["date"] = date
        precipitations_dict["prcp"] = prcp
        precipitations_dict["tobs"] = tobs
        all_precipitations.append(precipitations_dict)

    return jsonify(all_precipitations)



In [24]:
@app.route("/api/v1.0/stations")
def stations():
    # Create our session (link) from Python to the DB
    session = Session(engine)
    results = session.query(Station.station).all()
    session.close()

    # Create a list of stations
    all_stations = [station[0] for station in results]
    return jsonify(all_stations)



In [31]:
@app.route("/api/v1.0/tobs")
def tobs():
    session = Session(engine)

    stations_rows_count = session.query(Measurement.station, func.count(Measurement.station)).\
        group_by(Measurement.station).\
        order_by(func.count(Measurement.station).desc()).all()
    most_active_station = stations_rows_count[0][0]
    data = session.query(Measurement.date, Measurement.tobs).\
        filter(Measurement.station == most_active_station).\
        filter(Measurement.date >= '2016-08-23').\
        filter(Measurement.date <= '2017-08-23').all()
    
    session.close()

    tobsall = []
    for date, tobs in data:
        tobs_dict = {}
        tobs_dict["Date"] = date
        tobs_dict["Tobs"] = tobs
        tobsall.append(tobs_dict)

    return jsonify(tobsall)



AssertionError: View function mapping is overwriting an existing endpoint function: tobs

In [33]:
@app.route("/api/v1.0/<start>")
def get_t_start(start):
    session = Session(engine)
    result = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start).all()
    session.close()

    tobsall = []
    for min,avg,max in result:
        tobs_dict = {}
        tobs_dict["Min"] = min
        tobs_dict["Average"] = avg
        tobs_dict["Max"] = max
        tobsall.append(tobs_dict)

    return jsonify(tobsall)



In [34]:
@app.route("/api/v1.0/<start>/<end>")
def get_t_start_stop(start,stop):
    session = Session(engine)
    result = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start).filter(Measurement.date <= stop).all()
    session.close()

    tobsall = []
    for min,avg,max in result:
        tobs_dict = {}
        tobs_dict["Min"] = min
        tobs_dict["Average"] = avg
        tobs_dict["Max"] = max
        tobsall.append(tobs_dict)

    return jsonify(tobsall)


In [35]:
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\Eli\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
